In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
from datetime import date

In [2]:
# адрес начальной страницы для веб-скрейпинга
site = 'https://apteka.ru/'
url = 'https://apteka.ru/brand/'
# список брендов, данные которых необходимо получить
brands = ['oleos', 'miraroma']

In [3]:
# определяем кол-во страниц с товарами у бренда
def pages_count(url, brand):
    # получаем стартовую страницу для бренда
    response_start = requests.get(url+brand+'/')
    soup_start = BeautifulSoup(response_start.text, 'lxml')
    # определяем общее кол-во страниц с товарами (результат: количество+1)
    return len(soup_start.find('div', class_='Paginator').find_all('a'))
    
# извлекаем данные из карточек с несколькими вариантами
def card_variants(card, brand):
    # переходим на страницу товара
    sleep(3)
    result = []
    response_prod = requests.get(site + card.find('a').get('href'))
    soup = BeautifulSoup(response_prod.text, 'lxml')
    # определяем список с вариантами
    variants_prod = soup.find_all('a', class_='variantButton__button')
    
    for prod in variants_prod:
        name = prod.get('aria-label')
        price = prod.find('span', class_='moneyprice__roubles')
        # если "нет в наличии", т.е. не указана цена - не добавляем
        if price is not None:
            result.append([brand, name, int(price.text)])
    return result

# для карточек без вариантов
def card_no_variants(card, brand):
    name = card.find('span', class_='catalog-card__name emphasis').text
    price = card.find('span', class_='moneyprice__roubles')
    # если "нет в наличии", т.е. не указана цена - не добавляем
    if price is not None:
        return [brand, name, int(price.text)]

In [4]:
# список для хранения извлеченных данных
products = []
for brand in brands:
    count = pages_count(url, brand)
    print(brand)
    # собираем данные по товарам и ценам на каждой странице
    for i in range(1, count):
        print('page --', i)
        # получаем данные страницы
        sleep(3)
        response = requests.get(url + brand, params={'page':str(i)})
        soup = BeautifulSoup(response.text, 'lxml')
        # определяем список карточек с товарами
        cards = soup.find_all('div', class_='catalog-card card-flex')

        for card in cards:
            # проверяем текущую карточку на наличие других вариантов продукта
            if card.find('svg', class_='icon icon--buy') is not None:
                products.append(card_no_variants(card, brand))
            else:
                products += card_variants(card, brand)

oleos
page -- 1
page -- 2
page -- 3
page -- 4
page -- 5
page -- 6
page -- 7
miraroma
page -- 1
page -- 2


In [7]:
# убираем пустые значения из результата
products = list(filter(None, products))

# сохраняем в файл
df = pd.DataFrame(products, columns=['brand', 'name_product', 'price'])
df.to_csv('apteka_ru_'+str(date.today())+'.csv', sep=';')

file = pd.read_csv('apteka_ru_'+str(date.today())+'.csv', sep=';', index_col=0)
file.head()

,brand,name_product,price
0,oleos,Масло эфирное цитронелла 10 мл,162
1,oleos,Суперчистотел 5 мл,75
2,oleos,МАСЛО ЧЕРНОГО ТМИНА,736
3,oleos,Масло эфирное гвоздика 10 мл,124
4,oleos,Масло эфирное мята перечная 10 мл,133
